In [1]:
import cv2 
import numpy as np 
import mediapipe as mp
import time
import math
import pyautogui

In [10]:
mp_hands = mp.solutions.hands
hands_model = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

points = []
lines = []
circles = []

state = 'pen up'

capture = cv2.VideoCapture(0)

size = pyautogui.size()

screenx = size.width
screeny = size.height

def distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def midpoint(x1, y1, x2, y2):
    mx = (x1 + x2) / 2
    my = (y1 + y2) / 2
    return (mx, my)

while capture.isOpened():
    ret, frame = capture.read()
    if ret:
        frame = cv2.resize(frame, (screenx, screeny))

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = hands_model.process(image)
        image.flags.writeable = True

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                #mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                index_tipx = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * screenx
                index_tipy = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * screeny

                thumb_tipx = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x * screenx
                thumb_tipy = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * screeny

                middle_tipx = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * screenx
                middle_tipy = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * screeny
        else:
            index_tipx, index_tipy, thumb_tipx, thumb_tipy,middle_tipx, middle_tipy = 0,0,0,0,0,0

        point_dist_it = distance(index_tipx, index_tipy, thumb_tipx, thumb_tipy)
        point_dist_im = distance(index_tipx, index_tipy, middle_tipx, middle_tipy)

        if 10 <= point_dist_it <= 60 and state == 'pen down':
            points.append((int(index_tipx), int(index_tipy)))

        if 10 <= point_dist_it <= 60 and state == 'circle start':
            mx, my = midpoint(index_tipx, index_tipy, thumb_tipx, thumb_tipy)
            cc = (int(mx), int(my))
            state = 'circle draw'

        if 10 <= point_dist_it <= 60 and state == 'circle draw':
            mx, my = midpoint(index_tipx, index_tipy, thumb_tipx, thumb_tipy)
            co = (int(mx), int(my))
            rad = int(distance(cc[0], cc[1], co[0], co[1]))
            image = cv2.circle(image, cc, rad, (0, 255, 0), 3)

        if point_dist_it > 60 and state == 'circle draw':
            circles.append((cc, rad))
            state = 'circle start'

        if 10 <= point_dist_it <= 60 and state == 'line start':
            mx, my = midpoint(index_tipx, index_tipy, thumb_tipx, thumb_tipy)
            linep1 = (int(mx), int(my))
            state = 'line draw'

        if 10 <= point_dist_it <= 60 and state == 'line draw':
            mx, my = midpoint(index_tipx, index_tipy, thumb_tipx, thumb_tipy)
            linep2 = (int(mx), int(my))
            image = cv2.line(image, linep1, linep2, (0, 0, 255), 3)

        if point_dist_it > 60 and state == 'line draw':
            lines.append((linep1, linep2))
            state = 'line start'

        if (60 <= point_dist_im <= 90) and (630 <= index_tipx <= 800) and (55 <= index_tipy <= 130):
            state = 'line start'

        if (60 <= point_dist_im <= 90) and (155 <= index_tipx <= 410) and (55 <= index_tipy <= 130):
            state = 'circle start'

        if (60 <= point_dist_im <= 90) and (1050 <= index_tipx <= 1200) and (55 <= index_tipy <= 130):
            points.clear()
            lines.clear()
            circles.clear()

        if (60 <= point_dist_im <= 90) and (1450 <= index_tipx <= 1560) and (55 <= index_tipy <= 130):
            state = 'pen down'

        if (60 <= point_dist_im <= 90) and (1730 <= index_tipx <= 1900) and (55 <= index_tipy <= 130):
            state = 'pen up'

        for p in points:
            image = cv2.circle(image, (p[0], p[1]), 3, (255, 0, 0), 3)

        for l in lines:
            image = cv2.line(image, l[0], l[1], (0, 0, 255), 3)

        for c in circles:
            image = cv2.circle(image, c[0], c[1], (0, 255, 0), 3)

        flip_image = cv2.flip(image,1)
        #cv2.putText(flip_image, f'{index_tipx:.2f}, {index_tipy:.2f}', (10, 160), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(flip_image, f'State: {state}', (10, 140), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(flip_image, 'pen down', (400, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(flip_image, 'circle', (1600, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(flip_image, 'clear', (800, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(flip_image, 'line', (1200, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(flip_image, 'pen up', (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Hand Landmarks", flip_image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
    else:
        break

capture.release()
cv2.destroyAllWindows()
